In [6]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from P6_code.FinishedCode.importData import ImportEV, ImportWeather

In [8]:
data = ImportEV().getCaltech('2018-05-01', '2018-11-01')
data

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
270,5bc9160af9af8b0dad3c0690,0039,2018-05-01 06:23:18+00:00,2018-05-01 18:53:41+00:00,2018-05-01 10:17:07+00:00,13.410000,2_39_95_27_2018-05-01 13:23:18.374684,0002,CA-319,2-39-95-27,America/Los_Angeles,None,None
271,5bc9160af9af8b0dad3c0691,0039,2018-05-01 06:54:34+00:00,2018-05-01 16:15:35+00:00,2018-05-01 08:35:39+00:00,8.618000,2_39_79_380_2018-05-01 13:54:33.599590,0002,CA-489,2-39-79-380,America/Los_Angeles,None,None
272,5bc9160af9af8b0dad3c0692,0039,2018-05-01 07:46:04+00:00,2018-05-01 17:54:07+00:00,2018-05-01 16:57:02+00:00,13.969000,2_39_139_28_2018-05-01 14:46:04.441389,0002,CA-303,2-39-139-28,America/Los_Angeles,None,None
273,5bc9160af9af8b0dad3c0693,0039,2018-05-01 07:56:29+00:00,2018-05-01 14:52:45+00:00,2018-05-01 09:07:56+00:00,6.914000,2_39_79_378_2018-05-01 14:56:28.908623,0002,CA-326,2-39-79-378,America/Los_Angeles,None,None
274,5bc9160af9af8b0dad3c0694,0039,2018-05-01 08:14:50+00:00,2018-05-01 15:51:30+00:00,2018-05-01 13:18:12+00:00,13.785000,2_39_88_24_2018-05-01 15:14:50.443137,0002,CA-314,2-39-88-24,America/Los_Angeles,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13108,5c2e8550f9af8b13dab074d9,0039,2018-10-31 21:53:58+00:00,2018-10-31 22:32:43+00:00,2018-10-31 22:33:42+00:00,3.372000,2_39_90_26_2018-11-01 04:53:58.292201,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None
13109,5c2e8550f9af8b13dab074da,0039,2018-10-31 22:32:47+00:00,2018-10-31 22:53:40+00:00,2018-10-31 22:53:00+00:00,1.389000,2_39_90_26_2018-11-01 05:32:47.339653,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None
13110,5c2e8550f9af8b13dab074db,0039,2018-10-31 22:40:49+00:00,2018-11-01 00:57:28+00:00,2018-11-01 00:27:00+00:00,1.225597,2_39_78_365_2018-11-01 05:40:48.678916,0002,CA-321,2-39-78-365,America/Los_Angeles,None,None
13111,5c2e8550f9af8b13dab074dc,0039,2018-10-31 22:53:48+00:00,2018-10-31 23:24:48+00:00,2018-10-31 23:24:16+00:00,0.962000,2_39_90_440_2018-11-01 05:53:48.467907,0002,CA-500,2-39-90-440,America/Los_Angeles,None,None


In [ ]:
data = data[(data.connectionTime > '2018-05-01') & (data.connectionTime < '2018-11-01')]
data.reset_index(drop = True, inplace = True)
data

# Consumer Behavior

In [ ]:
ConnectionTime = pd.Series(range(len(data)), index=data["connectionTime"]).resample("h").count()
ct_mean = ConnectionTime.groupby(ConnectionTime.index.hour).mean()
ct_mean = ct_mean/sum(ct_mean)*100
ct_mean

In [ ]:
disconnectTime = pd.Series(range(len(data)), index=data["disconnectTime"]).resample("h").count()
dt_mean = disconnectTime.groupby(disconnectTime.index.hour).mean()
dt_mean = dt_mean/sum(dt_mean)*100
dt_mean

In [ ]:
ct_quartiles = ConnectionTime.groupby(ConnectionTime.index.hour).quantile([0.25, 0.50, 0.75])
ct_quartiles

In [ ]:
dt_quartiles = disconnectTime.groupby(disconnectTime.index.hour).quantile([0.25, 0.50, 0.75])
dt_quartiles

In [ ]:
dt_quartiles[0][0.50]

# Charging Time Stuff

In [ ]:
data["chargingTime"] = data["doneChargingTime"] - data["connectionTime"]
data["chargingTime"]

In [ ]:
#data["spaceID"].unique()
data["userID"].count()

In [ ]:
len(pd.unique(data["userID"]))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=ct_mean.index, y=ct_mean, error_y=dict(type='data', array=ct_quartiles.loc[:, 0.75] - ct_quartiles.loc[:, 0.50], arrayminus=ct_quartiles.loc[:, 0.50] - ct_quartiles.loc[:, 0.25]) ,name='Connection time'))
fig.add_trace(go.Bar(x=dt_mean.index, y=(-1)*dt_mean, error_y=dict(type='data', array=dt_quartiles.loc[:, 0.75] - dt_quartiles.loc[:, 0.50], arrayminus=dt_quartiles.loc[:, 0.50] - dt_quartiles.loc[:, 0.25]), name='Disconnection time'))

fig.update_layout(barmode='relative')
fig.show()

In [ ]:
data.describe()

In [ ]:
min(data["chargingTime"])

In [ ]:
data.sort_values(by="chargingTime")

In [ ]:
data["clusterID"] = len(data)*[39]
data["clusterID"].value_counts()

In [ ]:
data.loc["siteID"] = len(data) * [2]
data["siteID"].value_counts()

In [ ]:
data["spaceID"].value_counts()

In [ ]:
data[data["spaceID"] == 'CA-148']